# **Como subir un doc a Azure Blob Storage**
---

Lo primero es instalar el sdk.

In [ ]:
%pip install azure-storage-blob

Instalamos python-dotenv para cargar las variables de entorno.

In [ ]:
%pip install python-dotenv

### **Subir un Documento**

In [ ]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, ContentSettings
from azure.core.exceptions import ResourceExistsError, ResourceNotFoundError
import os
from dotenv import load_dotenv
load_dotenv()

# Parámetros de configuración
connect_str = BLOB_STORAGE_STRING
container_name = "Informacion Personal"  # Nombre del contenedor que creas en el blob
blob_name = "CURRICULUM.pdf"  # Nombre del archivo que subes al blob
local_file_path = "./Mario_Lamas_cv_2024-1.pdf"  # Ruta local del archivo a subir

# Crear un BlobServiceClient con la cadena de conexión
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# Crear un contenedor si no existe
try:
    container_client = blob_service_client.create_container(container_name)
    print(f"Contenedor '{container_name}' creado.")
except ResourceExistsError:
    print(f"El contenedor '{container_name}' ya existe.")

# Subir un archivo al Blob Storage
def upload_blob(local_file_path, container_name, blob_name):
    # Obtener el BlobClient para interactuar con un blob específico
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

    # Comprobar si el blob ya existe para evitar sobrescrituras
    try:
        blob_client.get_blob_properties()
        print(f"El blob '{blob_name}' ya existe en el contenedor '{container_name}'.")
        return
    except ResourceNotFoundError:
        print(f"El blob '{blob_name}' no existe. Procediendo a subir el archivo...")

    # Subir el archivo
    with open(local_file_path, "rb") as data:
        # overwrite=True para sobrescribir el blob si ya existe
        content_settings = ContentSettings(content_type="application/pdf",content_disposition='inline') # Tipo de contenido del archivo y disposición de descarga
        blob_client.upload_blob(data,content_settings=content_settings,overwrite=True)
        print(f"Archivo '{local_file_path}' subido a Blob Storage como '{blob_name}'.")

# Llamar a la función para subir el archivo
upload_blob(local_file_path, container_name, blob_name)


### **Como Listar los documentos que hay**
---

Para ver los blobs que hay dentro de un contenedor hacemos lo siguiente:

In [ ]:
# Listar blobs dentro de un contenedor
def list_blobs(container_name):
    container_client = blob_service_client.get_container_client(container_name)
    blobs_list = container_client.list_blobs()
    print(f"Lista de blobs en el contenedor '{container_name}':")
    for blob in blobs_list:
        print(f" - {blob.name}")

list_blobs(container_name)

# **Como descargar un documento**
---

Para descargar un blob hacemos lo siguiente.

In [ ]:
# Descargar un blob a la máquina local
def download_blob(container_name, blob_name, download_file_path):
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    
    print('URL del archivo: ',blob_client.url)

    with open(download_file_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())
        print(f"Blob '{blob_name}' descargado como '{download_file_path}'.")

# Descargar el archivo subido
download_file_path = os.path.join(os.getcwd(), "descargado_" + blob_name)
download_blob(container_name, blob_name, download_file_path)

### **Como eliminar un documento**
---

Para eliminar un blob hacemos lo siguiente.

In [ ]:

# Eliminar un blob del contenedor
def delete_blob(container_name, blob_name):
    try:
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
        blob_client.delete_blob()
        print(f"Blob '{blob_name}' eliminado del contenedor '{container_name}'.")
    except ResourceNotFoundError:
        print(f"El blob '{blob_name}' no existe en el contenedor '{container_name}'.")

# Opcional: eliminar el blob subido
delete_blob(container_name, blob_name)